In [193]:
import torch
import pandas as pd
import requests
from graphdriver.commons import data, results, setup
from graphdriver.utils import paths, cons

In [194]:
nt = ["genes","ppi"]

In [244]:
all_predicted_drivers = []
for c in setup.cancer_types():
    res = results.load_results(c, nt)
    ds = data.Dataset(c)
    d = ds.get_data()
    
    candidates_mask = torch.where(res.y == -1)[0]
    pred_all = torch.zeros(res.results[0].pred.shape)
    for r in res.results:
        pred_all += r.pred
    pred_all = pred_all/len(res.results)

    symbols = list(d.symbol_index_dict.keys())
    df = pd.DataFrame(pred_all, columns=["predictions"])
    df.index = symbols
    df = df.iloc[candidates_mask]
    df = df.sort_values(by=['predictions'], ascending=False).round(3)
    genes_cancer_drivers = df.iloc[:3].index.to_list()
    all_predicted_drivers.append(genes_cancer_drivers)


In [1]:
df = pd.DataFrame(all_predicted_drivers)
cancers = [c.upper() for c in setup.cancer_types()]
df.index = cancers

In [229]:
# print how many genes are unique
print(len(set(df.to_numpy().reshape(-1).tolist())))

37

In [2]:
df.to_latex("denovo.tex", escape=False)